In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as f

import numpy as np
import pandas as pd

from sklearn.metrics import auc, roc_curve, average_precision_score

import optuna

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
properties = ["RecordID", "Time"]
categorical = ["Gender", "MechVent"]
ordinal = ["GCS"]
target = "In-hospital_death"

### Train dataset

In [5]:
train = pd.read_parquet("../data/set-a.parquet") #TODO for Pascal
train = train.drop(columns=["ICUType"])
train

,RecordID,Time,Age,Gender,Height,Weight,Bilirubin,DiasABP,Lactate,MAP,...,pH,NIDiasABP,Platelets,SaO2,MechVent,GCS,PaO2,TroponinT,SysABP,In-hospital_death
0,132592,00:00,35.0,0.0,-1.0,71.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,132592,01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,132592,02:00,NaN,NaN,NaN,71.8,NaN,NaN,NaN,NaN,...,NaN,43.0,NaN,NaN,NaN,15.0,NaN,NaN,NaN,0
3,132592,03:00,NaN,NaN,NaN,71.8,NaN,NaN,NaN,NaN,...,NaN,53.0,287.0,NaN,NaN,NaN,NaN,0.15,NaN,0
4,132592,04:00,NaN,NaN,NaN,71.8,NaN,NaN,NaN,NaN,...,NaN,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195995,141565,44:00,NaN,NaN,NaN,NaN,NaN,81.0,NaN,103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,0
195996,141565,45:00,NaN,NaN,NaN,NaN,NaN,84.0,NaN,105.0,...,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,151.0,0
195997,141565,46:00,NaN,NaN,NaN,NaN,NaN,95.0,NaN,116.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,0
195998,141565,47:00,NaN,NaN,NaN,NaN,NaN,70.0,NaN,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131.0,0


In [6]:
# fill static variables (using mean but it should always be the first values since its the only one provided)
train["Age"] = train.groupby("RecordID").Age.transform(lambda x: x.fillna(x.mean(skipna=True)))
train["Gender"] = train.groupby("RecordID").Gender.transform(lambda x: x.fillna(x.mean(skipna=True)))
train

,RecordID,Time,Age,Gender,Height,Weight,Bilirubin,DiasABP,Lactate,MAP,...,pH,NIDiasABP,Platelets,SaO2,MechVent,GCS,PaO2,TroponinT,SysABP,In-hospital_death
0,132592,00:00,35.0,0.0,-1.0,71.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,132592,01:00,35.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,132592,02:00,35.0,0.0,NaN,71.8,NaN,NaN,NaN,NaN,...,NaN,43.0,NaN,NaN,NaN,15.0,NaN,NaN,NaN,0
3,132592,03:00,35.0,0.0,NaN,71.8,NaN,NaN,NaN,NaN,...,NaN,53.0,287.0,NaN,NaN,NaN,NaN,0.15,NaN,0
4,132592,04:00,35.0,0.0,NaN,71.8,NaN,NaN,NaN,NaN,...,NaN,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195995,141565,44:00,56.0,1.0,NaN,NaN,NaN,81.0,NaN,103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,0
195996,141565,45:00,56.0,1.0,NaN,NaN,NaN,84.0,NaN,105.0,...,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,151.0,0
195997,141565,46:00,56.0,1.0,NaN,NaN,NaN,95.0,NaN,116.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,0
195998,141565,47:00,56.0,1.0,NaN,NaN,NaN,70.0,NaN,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131.0,0


In [7]:
# store training mean and std for scaling the features in validation and test sets
cont_features = train.columns.difference(properties + categorical + [target])
cont_features_mean = train[cont_features].mean().to_dict()
cont_features_std = train[cont_features].std().to_dict()

In [8]:
# normalize continuous features
cont_features = train.columns.difference(properties + categorical + [target])
scaled_cont_features = train[cont_features].apply(lambda col: (col - cont_features_mean[col.name]) / cont_features_std[col.name])
scaled_cont_features[properties + [target]] = train[properties + [target]]
train = scaled_cont_features[properties + cont_features.to_list() + [target]]
train

,RecordID,Time,ALP,ALT,AST,Age,Albumin,BUN,Bilirubin,Cholesterol,...,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH,In-hospital_death
0,132592,00:00,NaN,NaN,NaN,-1.665689,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.462561,NaN,0
1,132592,01:00,NaN,NaN,NaN,-1.665689,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,132592,02:00,NaN,NaN,NaN,-1.665689,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.395347,NaN,NaN,NaN,NaN,-0.462561,NaN,0
3,132592,03:00,NaN,NaN,NaN,-1.665689,NaN,1.734332,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.386735,-0.022165,0.343136,-0.462561,NaN,0
4,132592,04:00,NaN,NaN,NaN,-1.665689,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.320021,NaN,-0.462561,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195995,141565,44:00,NaN,NaN,NaN,-0.469708,NaN,NaN,NaN,NaN,...,NaN,1.304081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
195996,141565,45:00,NaN,NaN,NaN,-0.469708,NaN,NaN,NaN,NaN,...,NaN,1.346996,0.234638,NaN,NaN,0.077120,NaN,NaN,NaN,0
195997,141565,46:00,NaN,NaN,NaN,-0.469708,NaN,NaN,NaN,NaN,...,NaN,2.033642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
195998,141565,47:00,NaN,NaN,NaN,-0.469708,NaN,NaN,NaN,NaN,...,NaN,0.488688,NaN,NaN,NaN,-0.220736,NaN,NaN,NaN,0


In [9]:
# columns one hot encoding
one_hot_feature_map = pd.get_dummies(train.columns, dtype=float).T.apply(lambda row: list(row), axis=1).to_dict()

In [10]:
# scale time
train["Scaled_Time"] = train.Time.apply(lambda t: int(t.split(":")[0]) / 48.0)
train

/var/folders/jp/s17dfmmd1lj9snp8qchr6nx00000gp/T/ipykernel_93725/277547138.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Scaled_Time"] = train.Time.apply(lambda t: int(t.split(":")[0]) / 48.0)


,RecordID,Time,ALP,ALT,AST,Age,Albumin,BUN,Bilirubin,Cholesterol,...,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH,In-hospital_death,Scaled_Time
0,132592,00:00,NaN,NaN,NaN,-1.665689,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.462561,NaN,0,0.000000
1,132592,01:00,NaN,NaN,NaN,-1.665689,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.020833
2,132592,02:00,NaN,NaN,NaN,-1.665689,NaN,NaN,NaN,NaN,...,NaN,-0.395347,NaN,NaN,NaN,NaN,-0.462561,NaN,0,0.041667
3,132592,03:00,NaN,NaN,NaN,-1.665689,NaN,1.734332,NaN,NaN,...,NaN,NaN,NaN,-0.386735,-0.022165,0.343136,-0.462561,NaN,0,0.062500
4,132592,04:00,NaN,NaN,NaN,-1.665689,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.320021,NaN,-0.462561,NaN,0,0.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195995,141565,44:00,NaN,NaN,NaN,-0.469708,NaN,NaN,NaN,NaN,...,1.304081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.916667
195996,141565,45:00,NaN,NaN,NaN,-0.469708,NaN,NaN,NaN,NaN,...,1.346996,0.234638,NaN,NaN,0.077120,NaN,NaN,NaN,0,0.937500
195997,141565,46:00,NaN,NaN,NaN,-0.469708,NaN,NaN,NaN,NaN,...,2.033642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.958333
195998,141565,47:00,NaN,NaN,NaN,-0.469708,NaN,NaN,NaN,NaN,...,0.488688,NaN,NaN,NaN,-0.220736,NaN,NaN,NaN,0,0.979167


In [11]:
results = []
for col in train.columns.difference(properties + [target] + ["Scaled_Time"]):
    _df = train[~train[col].isna()][properties + ["Scaled_Time", col]]
    _df["Triplet"] = _df.apply(lambda row: [row["Scaled_Time"]] + [one_hot_feature_map[col]] + [row[col]], axis=1)
    _df = _df.drop(columns=["Time", "Scaled_Time", col])
    results.append(_df)
    
triplets = pd.concat(results).sort_values(by="RecordID").reset_index(drop=True)
triplets

,RecordID,Triplet
0,132539,"[0.2916666666666667, [0.0, 0.0, 0.0, 0.0, 0.0,..."
1,132539,"[0.4166666666666667, [0.0, 0.0, 0.0, 0.0, 0.0,..."
2,132539,"[0.3333333333333333, [0.0, 0.0, 0.0, 0.0, 0.0,..."
3,132539,"[0.25, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,132539,"[0.16666666666666666, [0.0, 0.0, 0.0, 0.0, 0.0..."
...,...,...
1611251,142673,"[0.6666666666666666, [0.0, 0.0, 0.0, 0.0, 0.0,..."
1611252,142673,"[0.6875, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
1611253,142673,"[0.7083333333333334, [0.0, 0.0, 0.0, 0.0, 0.0,..."
1611254,142673,"[0.625, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."


In [12]:
train_triplets = triplets.groupby("RecordID").agg({"Triplet": list})
train_triplets = train_triplets.join(train.groupby("RecordID").agg({target: "first"}), on="RecordID").reset_index()
train_triplets

,RecordID,Triplet,In-hospital_death
0,132539,"[[0.2916666666666667, [0.0, 0.0, 0.0, 0.0, 0.0...",0
1,132540,"[[0.0625, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
2,132541,"[[0.8125, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3,132543,"[[0.6875, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
4,132545,"[[0.7916666666666666, [0.0, 0.0, 0.0, 0.0, 0.0...",0
...,...,...,...
3995,142665,"[[0.7916666666666666, [0.0, 0.0, 0.0, 0.0, 0.0...",0
3996,142667,"[[0.08333333333333333, [0.0, 0.0, 0.0, 0.0, 0....",0
3997,142670,"[[0.4375, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3998,142671,"[[0.375, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",1


In [13]:
MAX_LEN = train_triplets.Triplet.apply(len).max()
N_FEATURES = len(train_triplets.Triplet.iloc[0][0][1])

### Validation and testing datasets

In [ ]:
val = pd.read_parquet("../data/set-b.parquet") #TODO for Pascal
val = val.drop(columns=["ICUType"]) #TODO for Pascal

test = pd.read_parquet("../data/set-c.parquet")
test = test.drop(columns=["ICUType"])

# fill static variables (using mean but it should always be the first values since its the only one provided)
val["Age"] = val.groupby("RecordID").Age.transform(lambda x: x.fillna(x.mean(skipna=True)))
val["Gender"] = val.groupby("RecordID").Gender.transform(lambda x: x.fillna(x.mean(skipna=True)))

test["Age"] = test.groupby("RecordID").Age.transform(lambda x: x.fillna(x.mean(skipna=True)))
test["Gender"] = test.groupby("RecordID").Gender.transform(lambda x: x.fillna(x.mean(skipna=True)))

# normalize continuous features
scaled_cont_features = val[cont_features].apply(lambda col: (col - cont_features_mean[col.name]) / cont_features_std[col.name])
scaled_cont_features[properties + [target]] = val[properties + [target]]
val = scaled_cont_features[properties + cont_features.to_list() + [target]]

scaled_cont_features = test[cont_features].apply(lambda col: (col - cont_features_mean[col.name]) / cont_features_std[col.name])
scaled_cont_features[properties + [target]] = test[properties + [target]]
test = scaled_cont_features[properties + cont_features.to_list() + [target]]

# scale time
val["Scaled_Time"] = val.Time.apply(lambda t: int(t.split(":")[0]) / 48.0)
test["Scaled_Time"] = test.Time.apply(lambda t: int(t.split(":")[0]) / 48.0)

/var/folders/jp/s17dfmmd1lj9snp8qchr6nx00000gp/T/ipykernel_93725/2369070825.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["Scaled_Time"] = test.Time.apply(lambda t: int(t.split(":")[0]) / 48.0)


In [15]:
results = []
for col in val.columns.difference(properties + [target] + ["Scaled_Time"]):
    _df = val[~val[col].isna()][properties + ["Scaled_Time", col]]
    _df["Triplet"] = _df.apply(lambda row: [row["Scaled_Time"]] + [one_hot_feature_map[col]] + [row[col]], axis=1)
    _df = _df.drop(columns=["Time", "Scaled_Time", col])
    results.append(_df)
    
triplets = pd.concat(results).sort_values(by="RecordID").reset_index(drop=True)
val_triplets = triplets.groupby("RecordID").agg({"Triplet": list})
val_triplets = val_triplets.join(val.groupby("RecordID").agg({target: "first"}), on="RecordID").reset_index()

results = []
for col in test.columns.difference(properties + [target] + ["Scaled_Time"]):
    _df = test[~test[col].isna()][properties + ["Scaled_Time", col]]
    _df["Triplet"] = _df.apply(lambda row: [row["Scaled_Time"]] + [one_hot_feature_map[col]] + [row[col]], axis=1)
    _df = _df.drop(columns=["Time", "Scaled_Time", col])
    results.append(_df)
    
triplets = pd.concat(results).sort_values(by="RecordID").reset_index(drop=True)
test_triplets = triplets.groupby("RecordID").agg({"Triplet": list})
test_triplets = test_triplets.join(test.groupby("RecordID").agg({target: "first"}), on="RecordID").reset_index()

### Custom Dataset

In [16]:
from torch.utils.data import DataLoader, Dataset, SequentialSampler, WeightedRandomSampler
from torch.nn.utils.rnn import pad_sequence

class TripletDataset(Dataset):
    def __init__(self, df) -> None:
        self._df = df
        
    def __len__(self) -> int:
        return len(self._df)
    
    def __getitem__(self, idx) -> tuple:
        record =self._df.iloc[idx]
        
        time = torch.from_numpy(np.array([token[0] for token in record["Triplet"]]).astype(np.float32))
        feature = torch.from_numpy(np.array([token[1] for token in record["Triplet"]]).astype(np.float32))
        value = torch.from_numpy(np.array([token[2] for token in record["Triplet"]]).astype(np.float32))
        
        return time, feature, value, torch.tensor(record[target])

def pad_collate(batch) -> tuple:
    
    time, feature, value, labels = zip(*batch)
    time = list(time)
    feature = list(feature)
    value = list(value)
    labels = torch.tensor(labels, dtype=torch.long)
    
    time[0] = nn.ConstantPad1d((0, MAX_LEN - time[0].shape[0]), 0)(time[0])
    time = pad_sequence(time, batch_first=True, padding_value=0)
    
    feature[0] = nn.ConstantPad2d((0, 0, 0, MAX_LEN - feature[0].shape[0]), 0)(feature[0])
    feature = pad_sequence(feature, batch_first=True, padding_value=0)
    
    value[0] = nn.ConstantPad1d((0, MAX_LEN - value[0].shape[0]), 0)(value[0])
    value = pad_sequence(value, batch_first=True, padding_value=0)
    
    mask = (value != 0).float()

    return time, feature, value, labels, mask

### Create train, validation and test datasets

In [17]:
train_dataset = TripletDataset(train_triplets)

class_sample_count = train_triplets["In-hospital_death"].value_counts().to_numpy()

weight = 1.0 / class_sample_count
samples_weight = torch.from_numpy(weight[train_triplets["In-hospital_death"].to_numpy()]).double()
samples_weight

w_sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

val_dataset = TripletDataset(val_triplets)
test_dataset = TripletDataset(test_triplets)

### Example dataloader

In [18]:
train_dataloader = DataLoader(train_dataset, sampler=w_sampler, collate_fn=pad_collate, batch_size=2)

for batch in train_dataloader:
   times, features, values, labels, mask = batch
   print(times.shape, features.shape, values.shape, labels.shape, mask.shape)
   break

torch.Size([2, 677]) torch.Size([2, 677, 41]) torch.Size([2, 677]) torch.Size([2]) torch.Size([2, 677])


### Define transformer model

In [19]:
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [20]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, embed_dim: int, num_classes: int, n_heads: int, num_layers: int, dropout: float = 0.1) -> None:
        super().__init__()
        
        # TODO: THIS DOES NOT MAKE SENSE
        # Time embedding: Trigonometric encoding
        self.time_freq = 2 * torch.pi * torch.tensor([1, 2, 4], dtype=torch.float32).to(device)
        self.time_proj = nn.Linear(6, embed_dim)  # Project trig embedding
        
        # Feature embedding
        self.var_embed = nn.Linear(N_FEATURES, embed_dim, bias=False)
        
        # Value embedding
        self.value_proj = nn.Linear(1, embed_dim)
        
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=n_heads,
            dim_feedforward=4*embed_dim,
            dropout=dropout,
            activation=f.selu,
            batch_first=True
        )
        
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        self.dropout = nn.Dropout(dropout)
        
        self.output_layer = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(embed_dim, num_classes)
        )
        
        # outputs logits, there is no need for softmax (or log softmax), because CrossEntropyLoss
        # does this internally. If probabilities are needed, add f.log_softmax and use NLLoss.
        # self.output_layer = nn.Linear(embed_dim * MAX_LEN, num_classes)

    def forward(self, t: torch.Tensor, z: torch.Tensor, v: torch.Tensor, mask: torch.Tensor):
        # Time embedding: [sin(2πkt), cos(2πkt)]
        time_embed = torch.cat([
            torch.sin(t.unsqueeze(-1) * self.time_freq),
            torch.cos(t.unsqueeze(-1) * self.time_freq)
        ], dim=-1)  # Shape: [batch_size, seq_len, 6]
        
        time_embed = self.time_proj(time_embed)
        
        # Feature and value embeddings
        feature_embed = self.var_embed(z)  # [batch_size, seq_len, embed_dim] # .type(torch.long)
        val_embed = self.value_proj(v.unsqueeze(-1))  # [batch_size, seq_len, embed_dim]
        
        # print(time_embed.shape, feature_embed.shape, val_embed.shape)
        
        # Combine embeddings
        x = time_embed + feature_embed + val_embed
        
        # print(x.shape)
        
        # Apply Transformer with masking
        x = self.transformer_encoder(x, src_key_padding_mask=(mask == 0))  # Mask padded positions
        
        x = f.selu(x)
        x = x.permute(0, 2, 1)
        x = self.dropout(x)
        
        # print("dropout", x.shape)
        
        # Output
        # output = x #* padding_mask.unsqueeze(-1)  # zero-out padding embeddings
        # output = output.reshape(output.shape[0], -1)  # (batch_size, seq_length * d_model)
        
        # print("before output", output.shape)
        output = self.output_layer(x)  # (batch_size, num_classes)
        return output

In [21]:
model = TimeSeriesTransformer(
    embed_dim=64,
    num_classes=train["In-hospital_death"].nunique(),
    n_heads=4,
    num_layers=3
)

model.to(device)

/Users/asvozi/miniconda3/envs/ml4hc/lib/python3.11/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.activation_relu_or_gelu was not True
  warnings.warn(


TimeSeriesTransformer(
  (time_proj): Linear(in_features=6, out_features=64, bias=True)
  (var_embed): Linear(in_features=41, out_features=64, bias=False)
  (value_proj): Linear(in_features=1, out_features=64, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (output_layer): Sequent

In [22]:
train_dataloader = DataLoader(train_dataset, sampler=w_sampler, collate_fn=pad_collate, batch_size=32)
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), collate_fn=pad_collate, batch_size=32)

# Train the model
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-3)

n_batches = len(train_dataloader)
n_examples = len(train_dataloader.dataset) 
    
for epoch in range(20):
    print(f"Epoch {epoch}:")
    model.train()
    for i, batch in enumerate(train_dataloader):
       
        t, z, v, y, mask = batch
        t = t.to(device)
        z = z.to(device)
        v = v.to(device)
        mask = mask.to(device)
        y = y.to(device)
        
        pred = model(t, z, v, mask)
        loss = criterion(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()        
    
    model.eval()
    val_pred, val_true = [], []
    with torch.no_grad():
        for batch in val_dataloader:
            t, z, v, y, mask = batch
            t = t.to(device)
            z = z.to(device)
            v = v.to(device)
            mask = mask.to(device)
            
            logit = model(t, z, v, mask)           
            val_pred.extend(torch.argmax(logit.cpu(), dim=-1))
            val_true.extend(y.cpu())

    fpr, tpr, thresholds = roc_curve(np.array(val_true), np.array(val_pred) )
    print(auc(fpr, tpr))

Epoch 0:
0.5884241931777143
Epoch 1:
0.5512902590367379
Epoch 2:
0.6637857447716602
Epoch 3:
0.703638497652582
Epoch 4:
0.7243364030335862
Epoch 5:
0.7251961653370105
Epoch 6:
0.7496470665484749
Epoch 7:
0.7439693522440001
Epoch 8:
0.7520909255064184
Epoch 9:
0.7505335040546308
Epoch 10:
0.7555115072720707
Epoch 11:
0.7593363209560393
Epoch 12:
0.7398449555139697
Epoch 13:
0.7574403296234282
Epoch 14:
0.7599621622508946
Epoch 15:
0.7611584424964706
Epoch 16:
0.7379140812239404
Epoch 17:
0.7413613381923241
Epoch 18:
0.7541305525460454
Epoch 19:
0.7573746675859352


In [23]:
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), collate_fn=pad_collate, batch_size=32)

model.eval()
val_pred, val_true = [], []
with torch.no_grad():
    for batch in test_dataloader:
        t, z, v, y, mask = batch
        t = t.to(device)
        z = z.to(device)
        v = v.to(device)
        mask = mask.to(device)
        
        logit = model(t, z, v, mask).cpu()        
        val_pred.extend(torch.argmax(logit.cpu(), dim=-1))
        val_true.extend(y.cpu())

fpr, tpr, thresholds = roc_curve(np.array(val_true), np.array(val_pred))
print(f"AUC on test: {auc(fpr, tpr)}")

auprc = average_precision_score(np.array(val_true), np.array(val_pred))
print(f"AuPRC on test: {auprc}")

AUC on test: 0.7513371125376982
AuPRC on test: 0.3017367197728643
